## В данном проекте необходимо произвести сегментацию пользователей с помощью RFM-анализа. 
### Необходимо разделить пользователей по каждой категории на 3 сегмента, где 1 - худшая метка, а 3 - лучшая, и найти VIP-пользователей (имеющих сегмент 333), пользователей, которые много покупают, но давно этого не делали (131, 132, 133), пользователей, которые покупают часто, но на маленькую сумму и пользователей, которые "ушли в спячку".

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

### Загрузим и рассмотрим данные

In [2]:
# Загрузим данные и посмотрим на них
df = pd.read_csv('RFM_data.csv', parse_dates=['InvoiceDate'])
df.head()

,InvoiceNo,CustomerCode,InvoiceDate,Amount
0,C0011810010001,19067290,2020-09-01,1716.00
1,C0011810010017,13233933,2020-09-01,1489.74
2,C0011810010020,99057968,2020-09-01,151.47
3,C0011810010021,80007276,2020-09-01,146.72
4,C0011810010024,13164076,2020-09-01,104.00


In [3]:
# Посмотрим на типы данных и количество ненулевых значений
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 332730 entries, 0 to 332729
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   InvoiceNo     332730 non-null  object        
 1   CustomerCode  332730 non-null  object        
 2   InvoiceDate   332730 non-null  datetime64[ns]
 3   Amount        332730 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 10.2+ MB


In [4]:
# Посмотрим на описательные статистики для колонки Amount
df.Amount.describe()

count    332730.000000
mean       1239.029737
std        1747.362217
min       -8925.000000
25%         350.000000
50%         720.445000
75%        1491.507500
max      131874.000000
Name: Amount, dtype: float64

In [5]:
# Посмотрим за какой период у нас данные
print(f'Начальная и конечная дата в датасете: {df.InvoiceDate.min(), df.InvoiceDate.max()}')
print(f'Общее количество дней: {df.InvoiceDate.nunique()}')

Начальная и конечная дата в датасете: (Timestamp('2020-09-01 00:00:00'), Timestamp('2020-09-30 00:00:00'))
Общее количество дней: 30


### Проведем RFM-анализ

In [6]:
# Для каждого пользователя найдем количество дней после его последней покупки
last_date = df.InvoiceDate.max()
max_date = df.groupby('CustomerCode', as_index=False).agg({"InvoiceDate": lambda x: (last_date-x.max()).days}).rename(columns={'InvoiceDate': 'last_pc'})
max_date.head()

,CustomerCode,last_pc
0,2213019,6
1,2213071,7
2,2213081,9
3,2213082,10
4,2213128,8


In [7]:
# Для каждого пользователя найдем количество его покупок за все время 
pc_number = df.groupby('CustomerCode', as_index=False).agg(number=('InvoiceNo', 'count'))
pc_number.head()

,CustomerCode,number
0,2213019,1
1,2213071,1
2,2213081,1
3,2213082,2
4,2213128,2


In [8]:
# И наконец посчитаем сколько каждый клиент потратил денег
total_amount = df.groupby('CustomerCode', as_index=False).agg(total=('Amount', 'sum'))
total_amount.head()

,CustomerCode,total
0,2213019,2161.56
1,2213071,418.00
2,2213081,276.98
3,2213082,1215.96
4,2213128,981.23


In [9]:
# Посмотрим на топ-5 пользователей по принесенной выручке
total_amount.sort_values('total', ascending=False).head(5)

,CustomerCode,total
90095,13077590,379230.31
39928,15001294,368188.25
126923,15001294,343299.46
170110,99003061,296704.39
155451,19093056,287358.76


In [10]:
# Объединим все в один датафрейм
RFM = max_date.merge(pc_number, on="CustomerCode", how='left').merge(total_amount, on='CustomerCode', how='left')
RFM.head()

,CustomerCode,last_pc,number,total
0,2213019,6,1,2161.56
1,2213071,7,1,418.00
2,2213081,9,1,276.98
3,2213082,10,2,1215.96
4,2213128,8,2,981.23


In [11]:
# Теперь для каждого полученного значения поставим в соответствие метки по шкале от 1 до 3, воспользовавшись квантилями
RFM['R_score'] = pd.qcut(RFM['last_pc'], 3, labels=[3, 2, 1])
RFM['M_score'] = pd.qcut(RFM['total'], 3, labels=[1, 2, 3])
RFM.head()

,CustomerCode,last_pc,number,total,R_score,M_score
0,2213019,6,1,2161.56,3,2
1,2213071,7,1,418.00,2,1
2,2213081,9,1,276.98,2,1
3,2213082,10,2,1215.96,2,2
4,2213128,8,2,981.23,2,2


In [12]:
# Так как у нас очень много повторяющихся значений в столбце количества заказов, то напишем функцию для вычесления Frequency

score_one = RFM.number.quantile(0.33)
score_two = RFM.number.quantile(0.67)

def F_count(x):
    if x <= score_one:
        return 1
    elif x <= score_two:
        return 2
    else:
        return 3
    
RFM['F_score'] = RFM.number.apply(F_count)

In [13]:
RFM.head()

,CustomerCode,last_pc,number,total,R_score,M_score,F_score
0,2213019,6,1,2161.56,3,2,1
1,2213071,7,1,418.00,2,1,1
2,2213081,9,1,276.98,2,1,1
3,2213082,10,2,1215.96,2,2,2
4,2213128,8,2,981.23,2,2,2


In [14]:
# Теперь посчитаем полную RFM-сегментацию
RFM["RFM_score"] = RFM.R_score.astype('str') + RFM.F_score.astype('str') + RFM.M_score.astype('str')
RFM.head()

,CustomerCode,last_pc,number,total,R_score,M_score,F_score,RFM_score
0,2213019,6,1,2161.56,3,2,1,312
1,2213071,7,1,418.00,2,1,1,211
2,2213081,9,1,276.98,2,1,1,211
3,2213082,10,2,1215.96,2,2,2,222
4,2213128,8,2,981.23,2,2,2,222


In [15]:
# Создадим отдельный датафрейм
RFM_segmentation = RFM[['CustomerCode', 'R_score', 'F_score', 'M_score', 'RFM_score']]
RFM_segmentation.head()

,CustomerCode,R_score,F_score,M_score,RFM_score
0,2213019,3,1,2,312
1,2213071,2,1,1,211
2,2213081,2,1,1,211
3,2213082,2,2,2,222
4,2213128,2,2,2,222


In [16]:
# Посмотрим на количество VIP-пользователей
RFM_segmentation.query('RFM_score == "333"').CustomerCode.nunique()

17496

In [17]:
# Посмотрим на пользователей, которых нельзя потерять
RFM_segmentation.query('RFM_score in ("131", "132", "133")')

,CustomerCode,R_score,F_score,M_score,RFM_score
292,10001603,1,3,3,133
607,10007409,1,3,1,131
3782,13007721,1,3,3,133
4484,13012541,1,3,3,133
5598,13023028,1,3,3,133
...,...,...,...,...,...
173012,99098105,1,3,2,132
173029,99098776,1,3,3,133
173041,99099093,1,3,2,132
173048,99099196,1,3,3,133


In [18]:
# Посмотрим на пользователей, которые покупают часто, но на маленькую сумму
RFM_segmentation.query('RFM_score == "331"')

,CustomerCode,R_score,F_score,M_score,RFM_score
341,10002295,3,3,1,331
1076,10028159,3,3,1,331
1751,12115590,3,3,1,331
1837,12118282,3,3,1,331
2680,13000643,3,3,1,331
...,...,...,...,...,...
172461,99083430,3,3,1,331
172532,99085518,3,3,1,331
172643,99088331,3,3,1,331
172697,99089794,3,3,1,331


In [19]:
# И наконец посмотрим на пользователей, которые "ушли в спячку"
RFM_segmentation.query('RFM_score in ("111", "121", "122", "123", "133")')

,CustomerCode,R_score,F_score,M_score,RFM_score
5,2213163,1,1,1,111
17,2213378,1,1,1,111
38,3074261,1,1,1,111
46,3074417,1,1,1,111
87,3260223,1,1,1,111
...,...,...,...,...,...
173070,99099639,1,2,3,123
173073,99099769,1,1,1,111
173075,99099847,1,2,2,122
173077,99099878,1,2,3,123


### Вывод
Все задания были выполнены, RFM-анализ проведен успешно. Далее можно рассматривать остальные группы клиентов и принимать соответствующие решения для улучшения показателей.